In [101]:
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark import SparkConf, SQLContext
sparkSession = SparkSession.builder.appName("word-count").getOrCreate()

In [99]:
from pyspark.sql.functions import regexp_replace, trim, col, lower
def removePunctuation(column):
    return trim(lower(regexp_replace(column, '([^\s\w_]|_)+', ' '))).alias('sentence')
words = sparkSession.read.text('hdfs://localhost:9820/test/README.md').select(removePunctuation(col('value')))
words.show(truncate = False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|sentence                                                                                                                                                                                                                                                                                |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|apache spark                                                                                                                                          

In [90]:
from pyspark.sql.functions import split, explode

WordsSplitDF = (words.select(split(words.sentence, '\s+').alias('split')))
WordsSingleDF = (WordsSplitDF.select(explode(WordsSplitDF.split).alias('word')))
WordsDF = WordsSingleDF.where(WordsSingleDF.word != '')
WordsDF.show()
WordsDFCount = WordsDF.count()
print(WordsDFCount)

+----------+
|      word|
+----------+
|    apache|
|     spark|
|     spark|
|        is|
|         a|
|   unified|
| analytics|
|    engine|
|       for|
|     large|
|     scale|
|      data|
|processing|
|        it|
|  provides|
|      high|
|     level|
|      apis|
|        in|
|     scala|
+----------+
only showing top 20 rows

689


In [91]:
from pyspark.sql.functions import desc
def wordCount(wordListDF):
    return (wordListDF.groupBy('word').count())
WordsAndCountsDF = wordCount(WordsDF)
topWordsAndCountsDF = WordsAndCountsDF.orderBy("count", ascending=0)
topWordsAndCountsDF.show(689, truncate=False)

+------------------------+-----+
|word                    |count|
+------------------------+-----+
|spark                   |38   |
|the                     |25   |
|to                      |18   |
|https                   |16   |
|for                     |15   |
|run                     |13   |
|and                     |11   |
|apache                  |11   |
|a                       |10   |
|org                     |9    |
|hadoop                  |9    |
|building                |8    |
|example                 |8    |
|you                     |8    |
|html                    |7    |
|tests                   |7    |
|on                      |7    |
|is                      |7    |
|documentation           |6    |
|build                   |6    |
|can                     |6    |
|1000                    |6    |
|000                     |6    |
|in                      |5    |
|shell                   |5    |
|of                      |5    |
|master                  |5    |
|bin      

In [80]:
topWordsAndCountsDF.toPandas().to_csv('word-count.csv')